In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 2 : experimentation: get started with Vertex AI Feature Store

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_feature_store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
    
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_feature_store.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
        </a>
  </td>
    
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_feature_store.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
    
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 2 : experimentation: get started with Feature Store.

### Objective

In this tutorial, you learn how to use `Vertex AI Feature Store` when training and predicting with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Feature Store`

The steps performed include:

- Creating a Vertex AI `Featurestore` resource.
    - Creating `EntityType` resources for the `Featurestore` resource.
    - Creating `Feature` resources for each `EntityType` resource.
- Import feature values (entity data items) into `Featurestore` resource.
    - From a Cloud Storage location.
    - From a pandas DataFrame.
- Perform online serving from a `Featurestore` resource.
- Perform batch serving from a `Featurestore` resource.

### Dataset

The dataset used for this tutorial is the `Movie Recommendations` dataset. The version of the dataset you use in this tutorial is stored in a public Cloud Storage bucket, in Avro format.

This dataset is used to predict whether a person watches a movie or not.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage
- BigQuery

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing) and [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Installations

Install the following packages for further running this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the dependecies
! pip3 install --upgrade google-cloud-aiplatform google-cloud-bigquery pyarrow avro $USER_FLAG -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI, Compute Engine, Cloud Storage and Cloud Logging APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component,logging).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

1. **Click Create service account**.

2. In the **Service account name** field, enter a name, and click **Create**.

3. In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

4. Click Create. A JSON file that contains your key downloads to your local environment.

5. Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

Initialize Vertex AI and BigQuery clients.

In [ ]:
aiplatform.init(project=PROJECT_ID)
bqclient = bigquery.Client(project=PROJECT_ID)

## Introduction to Vertex AI Feature Store

Let's assume you have a recommendation model that predicts a coupon to print on the back of a cash register receipt. Now, if that model was trained only on single transaction instances (what was bought and how much), then (in the past) you used an Apriori algorithm.

But now we have historical data on the customer (say it's indexed by credit card number). Like total purchases to date, average purchase per transaction, frequency of purchase by product category, etc. We use this "enriched data" to train a recommender system.

Now it's time to do a live prediction. You get a transaction from the cash register, but all it has is the credit card number and this transaction. It does not have the enriched data the model needs. During serving, the credit card number is used as an index to Feature Store to get the enriched data needed for the model.

On the other hand, let's say the enriched data the model was trained on was timestamped on June 1st. The current transaction is from June 15th. Assume that the user has made other transactions between June 1st and 15th, and the enriched data has been continuously updated in Feature Store. But the model was trained on June 1st data. FeatureStore knows the version number and serves the June 1st version to the model (not the current June 15th). Otherwise, if you used June 15th data, you would have training-serving skew.

Another problem here is the data drift. Things change and suddenly one day, everybody is buying toilet paper! There is a significant change in the distribution of existing enriched data from the distribution that the deployed model was trained on. FeatureStore can detect changes/thresholds in distribution changes and trigger a notification for retraining the model.

Learn more about [Vertex AI Feature Store API](https://cloud.google.com/vertex-ai/docs/featurestore).

## Vertex AI Feature Store data model

Vertex AI Feature Store organizes data with the following 3 important hierarchical concepts:

        Featurestore -> EntityType -> Feature

- `Featurestore`: the place to store your features.
- `EntityType`: under a `Featurestore`, an `EntityType` describes an object to be modeled, real one or virtual one.
- `Feature`: under an `EntityType`, a `Feature` describes an attribute of the `EntityType`.

Learn more about [Vertex AI Feature Store data model](https://cloud.google.com/vertex-ai/docs/featurestore/concepts).

In the movie prediction dataset, you create a `Featurestore` resource called movies. This `Featurestore` resource has 2 entity types:
- `users`: The entity type has the `age`, `gender`, and `like genres` features.
- `movies`: The entity type has the `genres` and `average rating` features.

## Create a `Featurestore` resource

First, you create a `Featurestore` for the dataset using the `Featurestore.create()` method, with the following parameters:

- `featurestore_id`: The name of the feature store.
- `online_store_fixed_node_count`: Configuration settings for online serving from the feature store.
- `project`: The project ID.
- `location`: The location (region).

In [ ]:
# Represents featurestore resource path.
FEATURESTORE_NAME = "movies_" + TIMESTAMP

featurestore = aiplatform.Featurestore.create(
    featurestore_id=FEATURESTORE_NAME,
    online_store_fixed_node_count=1,
    project=PROJECT_ID,
    location=REGION,
)

print(featurestore)

### List your `Featurestore` resources

You can get a list of all your `Featurestore` resources in your project using the `Featurestore.list()` method.

In [ ]:
for featurestore in aiplatform.Featurestore.list():
    print(featurestore)

### Get a `Featurestore` resource

You can get a specifed `Featurestore` resource in your project using the `Featurestore()` initializer, with the following parameters:

- `featurestore_name`: The name for the `Featurestore` resource.
- `project`: The project ID.
- `location`: The location (region).

In [ ]:
featurestore = featurestore = aiplatform.Featurestore(
    featurestore_name=FEATURESTORE_NAME, project=PROJECT_ID, location=REGION
)
print(featurestore)

## Create entity types for your `Featurestore` resource

Next, you create the `EntityType` resources for your `Featurestore` resource using the `create_entity_type()` method, with the following parameters:

- `entity_type_id`: The name of the `EntityType` resource.
- `description`:  A description of the entity type.

In [ ]:
for name, description in [("users", "Users descrip"), ("movies", "Movies descrip")]:
    entity_type = featurestore.create_entity_type(
        entity_type_id=name, description=description
    )
    print(entity_type)

### Add `Feature` resources for your `EntityType` resources

Next, you create the `Feature` resources for each of the `EntityType` resources in your `Featurestore` resource using the `create_feature()` method, with the following parameters:

- `feature_id`: The name of the `Feature` resource.
- `description`: A description of the feature.
- `value_type`: The data type for the feature.

In [ ]:
def create_features(featurestore_name, entity_name, features):
    entity_type = aiplatform.EntityType(
        entity_type_name=entity_name, featurestore_id=featurestore_name
    )

    for feature in features:
        feature = entity_type.create_feature(
            feature_id=feature[0], description=feature[1], value_type=feature[2]
        )
        print(feature)


create_features(
    FEATURESTORE_NAME,
    "users",
    [
        ("age", "Age descrip", "INT64"),
        ("gender", "Gender descrip", "STRING"),
        ("liked_genres", "Genres descrip", "STRING_ARRAY"),
    ],
)

create_features(
    FEATURESTORE_NAME,
    "movies",
    [
        ("title", "Title descrip", "STRING"),
        ("genres", "Genres descrip", "STRING"),
        ("average_rating", "Ave descrip", "DOUBLE"),
    ],
)

### List your `Featurestore` resources

You can get a list of all your `Featurestore` resources in your project using the `Featurestore.list()` method.

In [ ]:
for featurestore in aiplatform.Featurestore.list():
    print(featurestore)

### Search `Feature` resources using a filter

You can narrow your search of `Feature` resources using the method `list_features()` and specifying a `filter` string.

In [ ]:
# Search by data type
features = []
for entity_type in featurestore.list_entity_types():
    features += entity_type.list_features(filter="value_type=DOUBLE")
print("By data type")
for feature in features:
    print(features)

# Search by data type
features = []
for entity_type in featurestore.list_entity_types():
    _ = entity_type.list_features()
    for feature in _:
        if feature.name == "title":
            print(type(feature))
            features += [feature]

print("By Name")
for feature in features:
    print(features)

### Search `Feature` resources using a query

You can narrow your search of `Feature` resources using the method `search()` and specifying a `query` filter.

In [ ]:
features = aiplatform.Feature.search(query="value_type=DOUBLE")
print("By data type")
for feature in features:
    print(features)

aiplatform.Feature.search(query="feature_id=title")
print("By Name")
for feature in features:
    print(features)

Define paths to the feature data.

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vertex-ai/feature-store/datasets/movie_prediction.csv"
)
FS_ENTITIES = {
    "users": "gs://cloud-samples-data/vertex-ai/feature-store/datasets/users.avro",
    "movies": "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro",
}

## Import the feature data into your `Featurestore` resource

Next, you import the feature data for your `Featurestore` resource. Once imported, you can use these feature values for online and offline (batch) serving.

### Data layout

Each imported `EntityType` resource data must have an ID. Also, each `EntityType` resource data item can optionally have a timestamp, sepecifying when the feature values were generated.

When importing, specify the following in your request:

- Data source format: BigQuery Table/Avro/CSV/Pandas Dataframe
- Data source URL
- Destination: featurestore/entity types/features to be imported

The feature values for `Movie Recommendations` dataset are in Avro format. The Avro schemas are as follows:

**Users entity**:

```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
 ```

**Movies entity**:

```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### Importing the feature values from Cloud Storage

You import the feature values for the `EntityType` resources using the `ingest_from_gcs()` method, with the following parameters:

- `entity_id_field`: The identifier name for the parent `EntityType` resource.
- `feature_ids`: A list of identifier names for `Feature` resources' data to add to the `EntityType` resource.
- `feature_time`: The field corresponding to the timestamp for the features being entered.
- `gcs_source_type`: The format of the imported data. Must be CSV or Avro.
- `gcs_source_uris`: A list of one or more Cloud Storage locations of the imported data files.

In [ ]:
entity_type = featurestore.get_entity_type("users")
response = entity_type.ingest_from_gcs(
    entity_id_field="user_id",
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    gcs_source_type="avro",
    gcs_source_uris=[FS_ENTITIES["users"]],
)
print(response)

entity_type = featurestore.get_entity_type("movies")
response = entity_type.ingest_from_gcs(
    entity_id_field="movie_id",
    feature_ids=["title", "genres", "average_rating"],
    feature_time="update_time",
    gcs_source_type="avro",
    gcs_source_uris=[FS_ENTITIES["movies"]],
)
print(response)

#### Delete the entity types and corresponding features and feature values

Now, in preparation to repeat the process of importing feature values but from a dataframe this time, you delete the existing entity types, and the corresponding content.

In [ ]:
entity_type = featurestore.get_entity_type("users")
entity_type.delete(force=True)
entity_type = featurestore.get_entity_type("movies")
entity_type.delete(force=True)

## Create entity types for your `Featurestore` resource

Next, you create the `EntityType` resources again for your `Featurestore` resource using the `create_entity_type()` method, with the following parameters:

- `entity_type_id`: The name of the `EntityType` resource.
- `description`:  A description of the entity type.

In [ ]:
for name, description in [("users", "Users descrip"), ("movies", "Movies descrip")]:
    entity_type = featurestore.create_entity_type(
        entity_type_id=name, description=description
    )
    print(entity_type)

### Add `Feature` resources for your `EntityType` resources

Further, you create the `Feature` resources again for each of the `EntityType` resources in your `Featurestore` resource using the `create_feature()` method, with the following parameters:

- `feature_id`: The name of the `Feature` resource.
- `description`: A description of the feature.
- `value_type`: The data type for the feature.

In [ ]:
def create_features(featurestore_name, entity_name, features):
    entity_type = aiplatform.EntityType(
        entity_type_name=entity_name, featurestore_id=featurestore_name
    )

    for feature in features:
        feature = entity_type.create_feature(
            feature_id=feature[0], description=feature[1], value_type=feature[2]
        )
        print(feature)


create_features(
    FEATURESTORE_NAME,
    "users",
    [
        ("age", "Age descrip", "INT64"),
        ("gender", "Gender descrip", "STRING"),
        ("liked_genres", "Genres descrip", "STRING_ARRAY"),
    ],
)

create_features(
    FEATURESTORE_NAME,
    "movies",
    [
        ("title", "Title descrip", "STRING"),
        ("genres", "Genres descrip", "STRING"),
        ("average_rating", "Ave descrip", "DOUBLE"),
    ],
)

Now, copy the `users` and `movies` data into avro files.

In [ ]:
GCS_USERS_AVRO_URI = FS_ENTITIES["users"]
GCS_MOVIES_AVRO_URI = FS_ENTITIES["movies"]

USERS_AVRO_FN = "users.avro"
MOVIES_AVRO_FN = "movies.avro"

! gsutil cp $GCS_USERS_AVRO_URI $USERS_AVRO_FN
! gsutil cp $GCS_MOVIES_AVRO_URI $MOVIES_AVRO_FN

#### Load Avro Files into pandas DataFrames

In [ ]:
from avro.datafile import DataFileReader
from avro.io import DatumReader


class AvroReader:
    def __init__(self, data_file):
        self.avro_reader = DataFileReader(open(data_file, "rb"), DatumReader())

    def to_dataframe(self):
        records = [record for record in self.avro_reader]
        return pd.DataFrame.from_records(data=records)


import pandas as pd

users_avro_reader = AvroReader(data_file=USERS_AVRO_FN)
users_source_df = users_avro_reader.to_dataframe()
print(users_source_df)

movies_avro_reader = AvroReader(data_file=MOVIES_AVRO_FN)
movies_source_df = movies_avro_reader.to_dataframe()
print(movies_source_df)

### Importing the feature values from DataFrame

You import the feature values for the `EntityType` resources using the `ingest_from_df()` method, with the following parameters:

- `entity_id_field`: The identifier name for the parent `EntityType` resource.
- `feature_ids`: A list of identifier names for `Feature` resources' data to add to the `EntityType` resource.
- `feature_time`: The field corresponding to the timestamp for the features being entered.
- `df_source`: The DataFrame containing the imported feature values.

In [ ]:
entity_type = featurestore.get_entity_type("users")
entity_type.ingest_from_df(
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    df_source=users_source_df,
    entity_id_field="user_id",
)

entity_type = featurestore.get_entity_type("movies")
entity_type.ingest_from_df(
    feature_ids=["average_rating", "title", "genres"],
    feature_time="update_time",
    df_source=movies_source_df,
    entity_id_field="movie_id",
)

## Vertex AI Feature Store serving

The Vertex AI Feature Store service provides the following two services for serving features from a `Featurestore` resource:

- Online serving - low-latency serving of small batches of features (prediction).

- Batch serving - high-throughput serving of large batches of features (training and prediction).

In [ ]:
def serve_features(featurestore, entity_name, features, id):
    entity_type = featurestore.get_entity_type(entity_name)
    return entity_type.read(entity_ids=[id], feature_ids=features)


features = serve_features(
    featurestore, "users", ["age", "gender", "liked_genres"], "alice"
)
print(features)

## Batch Serving

The Vertex AI Feature Store's batch serving service is optimized for serving large batches of features in real-time with high throughput, typically for training a model or batch prediction.

One can batch serve to the following destinations:

- BigQuery table
- Cloud Storage location
- Dataframe

### Output dataset

For batch serving, you use a BigQuery table for the output. First, you must create this output destination table.

In [ ]:
# Output dataset
DESTINATION_DATASET = f"movies_predictions_{TIMESTAMP}"

# Output table.
DESTINATION_TABLE = "training_data"  # @param {type:"string"}

DESTINATION_TABLE_URI = f"bq://{PROJECT_ID}.{DESTINATION_DATASET}.{DESTINATION_TABLE}"

dataset_id = f"{PROJECT_ID}.{DESTINATION_DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset = bqclient.create_dataset(dataset)
print("Created dataset:", dataset_id)

### Batch Read Feature Values

You batch serve entity data items to a BigQuery table using the `read_serve_to_bq()` method, with the following parameters:

- `bq_destination_output_uri`: The destination BigQuery table to receive the served features.
- `serving_feature_ids`: A dictionary of entity type and corresponding features to serve.
- `read_instances_uri`: A Cloud Storage location to read the entity data items from.

The output is stored in a BigQuery table.

In [ ]:
response = featurestore.batch_serve_to_bq(
    bq_destination_output_uri=DESTINATION_TABLE_URI,
    serving_feature_ids={
        "users": ["age", "gender", "liked_genres"],
        "movies": ["average_rating", "genres"],
    },
    read_instances_uri=IMPORT_FILE,
)

print(response)

## Cleaning up
### Delete a BigQuery dataset

Use the method `delete_dataset()` to delete a BigQuery dataset along with all its tables, by setting the parameter `delete_contents` to `True`.

In [ ]:
bqclient.delete_dataset(dataset, delete_contents=True)

### Delete a `Featurestore` resource

You can get a delete a specified `Featurestore` resource using the `delete()` method, with the following parameter:

- `force`: A flag indicating whether to delete a non-empy `Featurestore` resource.

In [ ]:
featurestore.delete(force=True)